In [72]:
import os
import logging
import importlib
import framework
importlib.reload(framework)
import test_classifier
importlib.reload(test_classifier)
import data_utils
importlib.reload(data_utils)
from framework import Usecase, DataCuration, FeatureEngineering, ModelEvaluator
import pandas as pd
import random
# Define some constants and configurations
logging.getLogger().setLevel(logging.INFO)

ACCESS_TOKEN = 'WUpGevbWC9lsnTW8quNUtmWRdAEM89'

Set up the usecase details

In [73]:
USECASE = 'ner'
USECASE_CONFIG = {
    'dataset': 'w2',
    'num_labels': 3,
    'labels_dict': {'person' : 0, 'org' : 1, 'none': 2},
     'cat_dict': {0: 'person', 1: 'org', 2: 'none'}
}

usecase = Usecase(USECASE)
usecase.setConfig(USECASE_CONFIG)

Set paths for datasets and goldens (local or ib, both work).
Specify configurations

In [74]:
DATAS = {
    ('ner', 'w2') : '/Users/ahsaasbajaj/Documents/Data/w2-instabase/flow/s2_map_records'
}

GOLDENS = {
  ('ner', 'w2') : '/Users/ahsaasbajaj/Documents/Data/w2-instabase/golden/goldens.csv'
}

DATASET = USECASE_CONFIG['dataset']
DATA = DATAS[(USECASE, DATASET)]
GOLDEN = GOLDENS[(USECASE, DATASET)]

GOLDEN_CONFIG = {
    'is_local': True,
    'index_field_name':'filename',
    'file_type': 'csv',
    'skip_first_row': True,
    'identifier': 'file'
}
DATASET_CONFIG = {
    'is_local': True, 
    'file_type': 'ibocr',
    'identifier': lambda path: os.path.basename(path).split('.ibocr')[0],
    'convert2txt': False
}

w2 = DataCuration(ACCESS_TOKEN, DATA, DATASET_CONFIG, GOLDEN, GOLDEN_CONFIG)   


INFO:root:Loading dataset from /Users/ahsaasbajaj/Documents/Data/w2-instabase/flow/s2_map_records
INFO:root:Loading goldens from /Users/ahsaasbajaj/Documents/Data/w2-instabase/golden/goldens.csv
Total files Goldens:  (154, 25)
Total files found in the source (142, 25)


In [75]:
PROCESSING_CONFIG = {
    'X_DIST_THRESHOLD': 100
}
CANDIDATES_FIELDS = {
    'person':'employee_name',
    'org':'employer_name'
}
w2.processIBOCR2candidatePhrases(DATASET_CONFIG, PROCESSING_CONFIG)
w2.compare_candidates_and_goldens(PROCESSING_CONFIG, CANDIDATES_FIELDS)

Generating candidates for 142 files
For X_DIST_THRESHOLD configuraion: 100
total files: 142
person names found in candidates: 131
org names found in candidates: 79



Generate test data from goldens (from actual persons and company names) or from ibocr (using candidate phrases generated by processIBOCR2candidatePhrases())

In [76]:
fe = FeatureEngineering(usecase, w2, CANDIDATES_FIELDS)
test_data_from_goldens = fe.generate_test_samples_from_goldens() # single dataframe
test_data_from_candidates = fe.generate_test_samples_from_candidates() # dict{'filename' : dataframe}

In [77]:
MODEL_TRAIN_DATA = 'w2'     # options for NER: (w2 or public)

MODELS = {
    ('ner', 'w2') : '/Users/ahsaasbajaj/Documents/Code/ner-hf/sequence-classification/w2/no-address/5/model.pt',
    ('ner', 'public') : '/Users/ahsaasbajaj/Documents/Code/ner-hf/sequence-classification/public/no-address/200/model.pt'
}

MODEL_CONFIG = {
    'model_file_or_path' : MODELS[(USECASE, MODEL_TRAIN_DATA)],
    'num_labels': USECASE_CONFIG['num_labels'],
}

EVAL_CONFIG = {
    'gpu': False,
    'use_goldens': False # True if test_data generated using generate_test_samples_from_goldens(), else False
}
test_data = test_data_from_candidates

# testing -- include as module alongwith support with sampling from df
# test_data = dict(random.sample(test_data.items(), 10))


In [78]:
model_evaluator = ModelEvaluator(usecase)
model_evaluator.set_config(MODEL_CONFIG, EVAL_CONFIG)

# Predictions (on test data generated from goldens)
output = model_evaluator.run_evaluation(test_data)

# Analyze results generated
if EVAL_CONFIG['use_goldens']:
    # output is a single df
    print('Sample outputs: ', output.head())
    model_evaluator.analyze_golden_result(output)
else:
    print(len(output.keys()))
    # output is a dictionary
    model_evaluator.analyze_overall_result(output, w2.golden, CANDIDATES_FIELDS)

ers/ahsaasbajaj/.cache/torch/transformers/90deb4d9dd705272dc4b3db1364d759d551d72a9f70a91f60e3a1f5e278b985d.9019d8d0ae95e32b896211ae7ae130d7c36bb19ccf35c90a9e51923309458f70
INFO:transformers.configuration_utils:Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_si